In [1]:
import re
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler
from math import log

import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta


from PyPDF2 import PdfFileWriter, PdfFileReader
from bs4 import BeautifulSoup

Run on local machine

In [2]:
#local
import glob
from wikipedia import wikipedia_awards
from utils import drop_based_on_occurance

Run on eikon codebook

In [118]:
#eikon
import eikon as ek
ek.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

2021-05-13 15:38:13,074 P[29288] [MainThread 140052970010432] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2021-05-13 15:38:13,078 P[29288] [MainThread 140052970010432] Port number was not identified, cannot send any request
2021-05-13 15:38:13,080 P[29288] [MainThread 140052970010432] Port number was not identified.
Check if Eikon Desktop or Eikon API Proxy is running.


EikonError: Error code -1 | Port number was not identified. Check if Eikon Desktop or Eikon API Proxy is running.

# Ideas
* do it with [intrinio](https://docs.intrinio.com/documentation/python/get_all_company_news_v2) [intrinio2](https://docs.intrinio.com/documentation/python/get_company_news_v2)
* do it with [yahoo](https://medium.com/@jamesbowden/extracting-esg-data-from-yahoo-finance-using-python-4834bc8db570) [yahoo2](https://rapidapi.com/apidojo/api/yahoo-finance1) [yahoo3](https://pypi.org/project/yfinance/)
* do these above also have esg scores?

# POC

1. Indexing and precomputed ESG scores
2. Blacklisting
3. PanamaPapers
4. Salaries
5. Awards
    * Select environmental, social, governmental awards???
    * Measure the sentiment attributed to particular news.

## Year selection

In [3]:
lower_boundary_date = "2020-01-01"
upper_boundary_date = "2021-01-01"

# Indexing

In [4]:
stock_exchanges = pd.read_excel("../data/scrapped_data.xlsx", sheet_name="exchanges", usecols=[0,1])

stock_exchanges.head(2)

,MIC,Name
0,ARCX,New York Stock Exchange Archipelago
1,FMTS,MTS France SAS


In [5]:
mics = stock_exchanges["MIC"].apply(lambda x: f"'{x}'")
mics = ",".join(mics)

In [6]:
instruments = f"""SCREEN(U(IN(Equity(active,public,countryprimaryquote))),
            IN(TR.ExchangeMarketIdCode,{mics}))"""

# dates = ["2018-01-01", "2019-01-01", "2020-01-01"]
# dates = ["2020-01-01"]
dates = [lower_boundary_date]

In [7]:
columns = ['Instrument', 'Market Identifier Code (MIC)', 'Company Common Name',
       'ESG Score', 'ESG Score Grade', "date"]

ESG_data = pd.DataFrame(columns=columns)
ESG_data

,Instrument,Market Identifier Code (MIC),Company Common Name,ESG Score,ESG Score Grade,date


In [9]:
for date in dates:
    params = {'params': {"SDate": date}}
    variables = ["TR.MIC", 'TR.CommonName', {"TR.TRESGScore":params}, {"TR.TRESGScoreGrade":params}]
    data = ek.get_data(instruments=instruments, fields=variables)
    if isinstance(data, tuple):
        data = data[0]
    data["date"] = date
    ESG_data = pd.concat([ESG_data, data])

NameError: name 'ek' is not defined

In [142]:
non_na_ESG_data = ESG_data[~ESG_data["ESG Score"].isna()]

In [10]:
non_na_ESG_data.sort_values(by="ESG Score", ascending=False)

,Instrument,Market Identifier Code (MIC),Company Common Name,ESG Score,ESG Score Grade,date
42283,AZNPy.BE,,AstraZeneca PLC,93.881462,A+,2021-01-01
48644,AZN.OQ,,AstraZeneca PLC,93.881462,A+,2021-01-01
39087,AZN.L,<NA>,AstraZeneca PLC,93.881462,A+,2021-01-01
7059,AZN.S,<NA>,AstraZeneca PLC,93.881462,A+,2021-01-01
58542,AZN.ST,<NA>,AstraZeneca PLC,93.881462,A+,2021-01-01
...,...,...,...,...,...,...
7556,300418.SZ,<NA>,Beijing Kunlun Tech Co Ltd,0.971407,D-,2021-01-01
41173,HPCO.BE,<NA>,Hallador Energy Co,0.929299,D-,2021-01-01
20089,28Y.F,<NA>,High Liner Foods Inc,0.696272,D-,2021-01-01
36465,CVM.A,<NA>,CEL-SCI Corp,0.532213,D-,2021-01-01


In [35]:
non_na_ESG_data.to_csv("../data/primary/ESG_Companies.csv", index=False)

NameError: name 'non_na_ESG_data' is not defined

In [8]:
ESG_data = pd.read_csv("../data/primary/ESG_Companies.csv")

In [9]:
companies_types = """(inc.?|corp.?|ag.?|ab.?|se.?|plc.?|a\.v\.v\..?|i\.l\..?|n\.v\..?|i\.l\..?|limited|ltd.?|s\.a\.|llc.?)"""

In [10]:
ESG_data["Company Common Name"] = ESG_data["Company Common Name"].apply(lambda x: re.sub(companies_types, "", x.lower()).strip())
ESG_data["Ticker"] = ESG_data["Instrument"].apply(lambda x:x.split(".")[0])
ESG_data = ESG_data.drop(columns="Instrument").drop_duplicates()

# Wage data

In [11]:
# load
data_russel = pd.read_excel('../data/scrapped_data.xlsx', sheet_name='PayRatiosRuss')
data_sp = pd.read_excel('../data/scrapped_data.xlsx', sheet_name='PayRatiosSP')

data_revenue_sp = pd.read_excel('../data/scrapped_data.xlsx', sheet_name="CompSP")
data_revenue_russel = pd.read_excel('../data/scrapped_data.xlsx', sheet_name="CompRussel")

data_russel.rename(columns={'Pay ratio to 1': 'PayRatio'}, inplace=True)
data_sp.rename(columns={'Pay ratio to 1': 'PayRatio'}, inplace=True)

# drop useless col
data_russel.drop(columns=['Pay RatioSort ascending'], inplace=True)
data_sp.drop(columns=['Pay RatioSort ascending'], inplace=True)

In [12]:
shape_sp, shape_russel = data_sp.shape, data_russel.shape
#
data_list = [data_russel, data_revenue_russel, data_sp, data_revenue_sp]

# compute minimum annual US salary
work_hours_annual = 2080
min_wage = 7.25
min_salary = min_wage * work_hours_annual

# create the columns with median to min ratio
data_russel['MedianMinRatio'], data_sp['MedianMinRatio'] = data_russel['MedianAnnualPay'] / min_salary, data_sp[
    'MedianAnnualPay'] / min_salary



In [13]:
# lowercase i strip na nazwach firm
data_russel['CompanyLower'] = [str.strip(str.lower(q)) for q in data_russel['Company']]
data_sp['CompanyLower'] = [str.strip(str.lower(q)) for q in data_sp['Company']]
# data_avg_salaries['CompanyLower'] = [str.strip(str.lower(q)) for q in data_avg_salaries['Company']]
data_revenue_sp['CompanyLower'] = [str.strip(str.lower(q)) for q in data_revenue_sp['Company Common Name']]

In [14]:
# lowercase i strip na nazwach firm
data_russel['CompanyLower'] = [str.strip(str.lower(q)) for q in data_russel['Company']]
data_sp['CompanyLower'] = [str.strip(str.lower(q)) for q in data_sp['Company']]
# data_avg_salaries['CompanyLower'] = [str.strip(str.lower(q)) for q in data_avg_salaries['Company']]
data_revenue_sp['CompanyLower'] = [str.strip(str.lower(q)) for q in data_revenue_sp['Company Common Name']]

# drop column
# data_avg_salaries.drop('Company', inplace=True, axis=1)
data_russel.drop('Company', inplace=True, axis=1)
data_sp.drop('Company', inplace=True, axis=1)

# cast Tickers to string and sort dataframe
for each in data_list:
    each['Ticker'] = [str(q) for q in each['Ticker']]
    each.sort_values(by='Ticker', inplace=True)


In [15]:
# merge with data from refinitiv
data_russel_no_avg = data_russel.merge(data_revenue_russel, on='Ticker')
data_sp_no_avg = data_sp.merge(data_revenue_sp, on='Ticker')

# check shapes with duplicates
shape_russel_dup = data_russel_no_avg.shape
shape_sp_dup = data_sp_no_avg.shape

# drop duplicates
data_russel_no_avg.drop_duplicates(inplace=True)
data_sp_no_avg.drop_duplicates(inplace=True)

# check shape after drop
shape_russel = data_russel_no_avg.shape
shape_sp = data_sp_no_avg.shape

#put the data into structure (albeit a trivial one)
merged_data_list = [data_russel_no_avg, data_sp_no_avg]

In [16]:
#prepare data for topsis
data_for_topsis = []

for each in merged_data_list:
    # compute efficiency wage component
    each['Comp/Reve'] = each['Labor Expenses'] / each['Revenue BA']
    # inverse pay ratio to get a stimulant
    each['PayRatioStim'] = 1 / each['PayRatio']

    # cut out the companies with 0 or negative revenues or labour expenses
    each = each.loc[(each['Revenue BA'] > 0) & (each['Labor Expenses'] > 0)]

    # prepare table for aggregation
    table_for_topsis = each[['Ticker', 'MedianAnnualPay', 'PayRatioStim', 'MedianMinRatio', 'Comp/Reve']]


    table_for_topsis.set_index('Ticker', drop=True, inplace=True)
    data_for_topsis.append(table_for_topsis)


In [17]:
scaled_data = []

for i in range(len(data_for_topsis)):

    #
    each = data_for_topsis[i]
    indeks = data_for_topsis[i].index
    columns = data_for_topsis[i].columns

    col_list = []

    for column in each.columns:

        col_to_scale = each[str(column)].values.reshape(-1, 1)
        #logarithm on the column to get rid of the outliers
        loged_col = np.array([log(q) for q in col_to_scale]).reshape(-1, 1)

        #scale by min max formula
        transformed = MinMaxScaler().fit_transform(loged_col)

        listed_transformed = transformed.reshape(1, -1).tolist()
        col_list.append(listed_transformed[0])

    #prepare scaled df
    scaled_df = pd.DataFrame(col_list).transpose()
    scaled_df.index = indeks
    scaled_df.columns = columns

    scaled_data.append(scaled_df)

#look at the data
scaled_df_rus = scaled_data[0]
scaled_df_sp = scaled_data[1]


In [18]:
rus_dist_list = []
sp_dist_list = []

#compute distances separately for russel and sp
for i in range(len(scaled_df_rus.index)):
    row = scaled_df_rus.iloc[i, :].values

    #compute euclidean distance from the best theoretical best object (since we used min max scaler the best one will be (1,1,1,1))
    dist = distance.euclidean(np.ndarray([1, 1, 1, 1]), row)
    rus_dist_list.append(dist)

for i in range(len(scaled_df_sp.index)):
    row = scaled_df_sp.iloc[i, :].values

    dist = distance.euclidean(np.ndarray([1, 1, 1, 1]), row)
    sp_dist_list.append(dist)

#prepare distance dfs
dist_df_rus = pd.DataFrame(rus_dist_list)
dist_df_sp = pd.DataFrame(sp_dist_list)

dist_df_rus.index = scaled_df_rus.index
dist_df_sp.index = scaled_df_sp.index

dist_df_rus.columns = ['Distance']
dist_df_sp.columns = ['Distance']

#put the dfs in a structure
dist_df_list = [dist_df_rus, dist_df_sp]

In [19]:
agregate_vars_df_list = []

for each in dist_df_list:
    #compute avg and std of distances
    dist_avg = np.average(each['Distance'])
    dist_std = np.std(each['Distance'])

    # compute the aggregate variable
    agregate_vars = [(1 - (q / dist_avg + (2 * dist_std))) for q in each['Distance']]

    #prepare the df
    agregate_vars_df = pd.DataFrame(agregate_vars)
    agregate_vars_df.index = each.index
    agregate_vars_df.columns = ['AggregateValue']

    agregate_vars_df_list.append(agregate_vars_df)

#prepare final dfs
final_rus = agregate_vars_df_list[0]
final_sp = agregate_vars_df_list[1]

final_rus.reset_index(inplace=True)
final_sp.reset_index(inplace=True)

#check the shapes (to see if we didnt loose anyone on our way here
final_shapes = [final_rus.copy().shape, final_sp.copy().shape]

# company_names_sp = data_russel[['Ticker', 'CompanyLower']]
# company_names_russel = data_sp[['Ticker', 'CompanyLower']]


In [20]:
wages_data = final_rus

In [21]:
# wages_data = pd.read_csv('../data/wages/fair_wage_index_russel_3000.csv')

In [22]:
ESG_data = pd.merge(ESG_data, wages_data, on="Ticker", how="inner")

ESG_data = ESG_data.rename(columns={"AggregateValue":"WagesAggregateValue"})
ESG_data

,Market Identifier Code (MIC),Company Common Name,ESG Score,ESG Score Grade,date,Ticker,WagesAggregateValue
0,NaN,finet,50.356719,B-,2021-01-01,FN,-0.830556
1,NaN,codexis,57.657252,B-,2021-01-01,CDXS,-0.191844
2,NaN,sps commerce,53.880855,B-,2021-01-01,SPSC,-0.344557
3,NaN,green dot,28.248551,C-,2021-01-01,GDOT,-0.244501
4,NaN,neophotonics,25.580474,C-,2021-01-01,NPTN,-0.610388
...,...,...,...,...,...,...,...
944,NaN,maxlinear,25.003692,C-,2021-01-01,MXL,-0.276712
945,NaN,calix,31.664495,C-,2021-01-01,CALX,-0.221678
946,NaN,first interstate bancsystem,44.353486,C+,2021-01-01,FIBK,-0.336117
947,NaN,ss&c technologies holdings,18.720073,D+,2021-01-01,SSNC,-0.421993


# Blacklisting

### Defence companies anti corruption index

In [23]:
with open("../data/lists/csv/defence_companies_anti_corruption_index.csv", "r", encoding="unicode_escape") as f:
    file = f.readlines()

anti_corruption_data = "".join(file[5:])

with open("../data/lists/csv/defence_companies_anti_corruption_index_preprocessed.csv", "w", encoding="utf-8") as f:
    f.write(anti_corruption_data)

In [24]:
anti_corruption_data = pd.read_csv("../data/lists/csv/defence_companies_anti_corruption_index_preprocessed.csv", header=[0,1,2])

In [25]:
anti_corruption_data_filtered = anti_corruption_data[[anti_corruption_data.columns[0], anti_corruption_data.columns[-9]]].copy()
anti_corruption_data_filtered.columns = ["Company Common Name", "anti corruption score"]
anti_corruption_data_filtered["anti corruption score normalized"] = anti_corruption_data_filtered["anti corruption score"] / anti_corruption_data_filtered["anti corruption score"].max()

list_anti_corruption = anti_corruption_data_filtered[anti_corruption_data_filtered["anti corruption score normalized"]<.5]["Company Common Name"]
list_anti_corruption = list(map(lambda x: x.lower(), list_anti_corruption))

In [26]:
ESG_data = drop_based_on_occurance(ESG_data, variable="Company Common Name", condition=list_anti_corruption)
len(ESG_data)

940

### Testing on animals

In [27]:
animals_testing = PdfFileReader(open("../data/lists/pdf/companies-do-test.pdf", "rb"))

In [28]:
text = []
for page_idx in range(animals_testing.getNumPages()):
    text.append(animals_testing.getPage(page_idx).extractText())

In [29]:
animals_testing = "".join(text).lower()

In [30]:
ESG_data = drop_based_on_occurance(ESG_data, variable="Company Common Name", condition=animals_testing)
len(ESG_data)

930

### Fossil fuels

In [31]:
fossil_fuels = pd.read_csv("../data/lists/csv/fossil-fuels.csv")

fossil_fuels = list(fossil_fuels["Company Name"].apply(lambda x: x.lower()))

In [32]:
ESG_data = drop_based_on_occurance(ESG_data, variable="Company Common Name", condition=fossil_fuels)

In [33]:
len(ESG_data)

900

# Whitelisting

### Working for regulatory change

In [34]:
regulatory_change = PdfFileReader(open("../data/lists/pdf/companies-working-for-regulatory-change.pdf", "rb"))

In [35]:
regulatory_change_text = []
for page_idx in range(regulatory_change.getNumPages()):
    regulatory_change_text.append(regulatory_change.getPage(page_idx).extractText())

In [36]:
regulatory_change = "".join(regulatory_change_text).lower()

In [37]:
ESG_data["regulatory_change"] = ESG_data["Company Common Name"].apply(lambda x: int(x in regulatory_change))

ESG_data["regulatory_change"].sum()

0

### Cruelty free companies

In [38]:
cruelty_free = PdfFileReader(open("../data/lists/pdf/cruelty-free-companies-united-states.pdf", "rb"))

In [39]:
cruelty_free_text = []
for page_idx in range(cruelty_free.getNumPages()):
    cruelty_free_text.append(cruelty_free.getPage(page_idx).extractText())

In [40]:
cruelty_free = "".join(cruelty_free_text).lower()

In [41]:
ESG_data["cruelty_free_list"] = ESG_data["Company Common Name"].apply(lambda x: int(x in cruelty_free))

ESG_data["cruelty_free_list"].sum()

11

# Panama papers

In [42]:
files = [x for x in glob.glob("../data/paradise_papers/**", recursive=True) if "entity" in x or "intermediary" in x]

In [43]:
panama_companies = pd.DataFrame(columns=["name"], dtype=str)

In [44]:
for file in files:
    temp = pd.read_csv(file, usecols=["name"])
    panama_companies = pd.concat([panama_companies, temp])
len(panama_companies)

810869

In [45]:
panama_companies["name"] = panama_companies["name"].astype(str)

panama_companies["name"] = panama_companies["name"].apply(lambda x: re.sub(companies_types, '', x.lower()).strip())

panama_companies["name"] = panama_companies["name"].apply(lambda x: re.sub(r'\([^)]*\)', '', x.lower()).strip())

panama_companies = list(panama_companies.reset_index(drop=True)["name"])

In [46]:
ESG_data = drop_based_on_occurance(ESG_data, variable="Company Common Name", condition=panama_companies)
ESG_data = ESG_data.reset_index(drop=True)
len(ESG_data)

679

# Awards data

* [Another list -> environmental awards](https://en.wikipedia.org/wiki/List_of_environmental_awards)


* [sentiment](https://developers.refinitiv.com/en/article-catalog/article/introduction-news-sentiment-analysis-eikon-data-apis-python-example)
* [sentiment2](https://www.refinitiv.com/perspectives/future-of-investing-trading/news-sentiment-analysis-with-eikon-data-apis/)


=================================================
<br>Pretrained etc.
* [OpenAI](https://github.com/openai/generating-reviews-discovering-sentiment)
* [Pytorch implementation](https://github.com/guillitte/pytorch-sentiment-neuron)
* [Off shelf](http://nlp.town/blog/off-the-shelf-sentiment-analysis/)
* [vader](https://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html)
* [fasttext](https://fasttext.cc/)
* [TextBlob](https://medium.com/@b.terryjack/nlp-pre-trained-sentiment-analysis-1eb52a9d742c)


In [47]:
wiki_awards = wikipedia_awards()

scrapped_awards = pd.read_excel("../data/scrapped_data.xlsx", sheet_name="awards", usecols=[0,1,2])

scrapped_awards["international"] = scrapped_awards["international"].apply(lambda x: "I" in x or "R" in x)

awards = pd.concat([wiki_awards, scrapped_awards]).drop_duplicates().reset_index(drop=True)
awards["award"] = awards["award"].apply(lambda x: x.lower())

In [48]:
awards.to_csv("../data/awards/awards.csv", index=False)

In [49]:
awards = pd.read_csv("../data/awards/awards.csv")
awards

,international,country,award
0,True,NaN,applause award
1,True,NaN,arthur b. guise medal
2,True,NaN,golden ticket awards
3,True,NaN,hotelier of the century
4,True,NaN,seven stars luxury hospitality and lifestyle a...
...,...,...,...
659,True,NaN,topgun designation
660,True,EU,best of european business
661,True,NaN,public eye on davos
662,True,EU,100 best workplaces in europe


In [50]:
len(awards[awards["international"]==True])

466

In [51]:
ESG_data.to_csv("../data/ESG_after_awards.csv", index=False)

In [52]:
ESG_data

,Market Identifier Code (MIC),Company Common Name,ESG Score,ESG Score Grade,date,Ticker,WagesAggregateValue,regulatory_change,cruelty_free_list
0,NaN,codexis,57.657252,B-,2021-01-01,CDXS,-0.191844,0,0
1,NaN,sps commerce,53.880855,B-,2021-01-01,SPSC,-0.344557,0,0
2,NaN,neophotonics,25.580474,C-,2021-01-01,NPTN,-0.610388,0,0
3,NaN,first american financial,45.545963,C+,2021-01-01,FAF,-0.397892,0,0
4,NaN,cboe global markets,47.740064,C+,2021-01-01,CBOE,-0.210807,0,0
...,...,...,...,...,...,...,...,...,...
674,NaN,quinstreet,21.663170,D+,2021-01-01,QNST,-0.431739,0,0
675,NaN,sata technologies holding,59.879600,B,2021-01-01,ST,-0.851722,0,0
676,NaN,first interstate bancsystem,44.353486,C+,2021-01-01,FIBK,-0.336117,0,0
677,NaN,ss&c technologies holdings,18.720073,D+,2021-01-01,SSNC,-0.421993,0,0
